# `where` are the `layerDefs`?

Currently there is no official api published on how to further filter the result list returned by `/identify`. There is a working implementation using the query parameter `where`, which is used internally by the viewer, and maybe also externally (to be clarified). This query parameter is however not documented. Upon two requests from clients there have been made attempts to improve and stabilize the possibility to filter. In the current WIP, two query params are used that use exactly the same parsing logic in the background, but different formats in the query, `where` and `layerDefs`. Ideally, we'd just making one of them publicly available (and documented). You find below on how to use them.

Note: Since `where` might sound too intriguing not to try any sqlinjection stuff, we could rename it to `filter`. That's what it actually does. 

In [191]:
import requests
import pprint
import urllib.parse

# BRANCH = 'mom_extend_find'
# BRANCH = 'ltrea_extend_find'
BRANCH = 'ltboc'


def get_url(params):
    """ params needs to be a dict of the form
    where = {
        "branch":'',
        "paramName":'where',
        "paramValue":"plzo+=+'8302 Kloten'"
    }
    """
    _url = """https://mf-chsdi3.dev.bgdi.ch/
{branch}/
rest/services/all/MapServer/
identify?
geometry=2577070,1184860
&mapExtent=0,0,100,100
&imageDisplay=100,100,100
&geometryFormat=geojson
&geometryType=esriGeometryPoint
&lang=de
&layers=all:{layer}
&returnGeometry=false
&tolerance=500000
&sr=2056
&{paramName}={paramValue}
    """
    url = _url.format(**params).strip().replace('\n','')
    print("\n=================\nrequesting {}".format(url))
    response = requests.get(url)
    if response.status_code == 200:
        results = response.json().get('results', [])
        print("--> success! {} results".format(len(results)))
        pprint.pprint(next(iter(results), []))
    else:
        print("--> example :{}: returned error code {}: {}".format(example, response.status_code, response.text))

def encode_param(query):
    return urllib.parse.quote(query)

# {"ch.swisstopo.amtliches-strassenverzeichnis":"plzo%20ilike%20'%Basel%'","ch.swisstopo.amtliches-strassenverzeichnis":"label%20ilike%20'%Hauptstrasse%'"}

res=requests.get("https://mf-chsdi3.dev.bgdi.ch/mom_extend_find/rest/services/all/MapServer/ch.swisstopo.amtliches-strassenverzeichnis")
pprint.pprint(res.json())

examples = {
    "ch.bazl.luftfahrthindernis": [
        "bgdi_activesince <= '2019-07-30'"
    ],
    "ch.swisstopo.amtliches-strassenverzeichnis": [
        "plzo ilike '%Olten%' and label ilike '%Studerweg%'",
        "plzo ilike '%Olten%' and label ilike '%Studerwe'", # should return no results
        "plzo ilike '%Olten%' or label ilike '%Studerweg%'",
        "plzo ilike '%Olten%' OR label ilike '%Studerweg%'",
        "label = 'Studerweg' and plzo not ilike '%Olten%'",
        "plzo = '8302 Kloten'",
        "plzo ilike '%Murten%'",
        "plzo ilike '%Basel%'",
        "plzo ilike '8302%'",
        "blah ilike '8302%'", # should not work
        "gdename ilike 'sel%'",
        "label ilike '%Hauptstrasse%'",
        "label ilike 'Im Heimatla%'", # 'Im Heimatland' in Basel
        "gdenr > 2701 and gdenr < 2704",
        "gdenr > 2701 and gdenr < 2704 and gdename = 'Seltisberg'", # should not return any results
    ],
    "ch.bfe.ladestellen-elektromobilitaet": [
        "IsOpen24Hours = true",
        "IsOpen24Hours is true",
        "QueryAuthenticationModes ilike '%nfc%'", 
        "Longitude > 7.476",
        "QueryAuthenticationModes ilike '%nfc%' and Longitude > 7.476",
    ],
#     "ch.bav.haltestellen-oev,ch.swisstopo.amtliches-strassenverzeichnis"
}


{'fields': [{'alias': 'PLZ / Ortschaft',
             'name': 'plzo',
             'type': 'VARCHAR',
             'values': ['4052 Basel',
                        '4057 Basel',
                        '4058 Basel',
                        '4059 Basel',
                        '4125 Riehen']},
            {'alias': 'Objekttyp',
             'name': 'type',
             'type': 'VARCHAR',
             'values': ['Platz', 'Strasse']},
            {'alias': 'BFS-Nr.',
             'name': 'gdenr',
             'type': 'INTEGER',
             'values': [2701, 2702, 2703]},
            {'alias': 'Gemeindename',
             'name': 'gdename',
             'type': 'VARCHAR',
             'values': ['Basel', 'Bettingen', 'Riehen']},
            {'alias': 'Strassenname',
             'name': 'label',
             'type': 'VARCHAR',
             'values': ['Grendelgasse',
                        'Hohlweg',
                        'Sperberweg',
                        'Sperrweglein',
           

In [133]:
sqlinj = {
    "ch.swisstopo.amtliches-strassenverzeichnis": ["1=1"]
}
for layer, layer_examples in sqlinj.items():
    for example in layer_examples:
        where_params = {
            "branch":BRANCH,
            "paramName":'where',
            "layer":layer,
            "paramValue":encode_param(example)
        }
        get_url(where_params)

# identifyurl = """https://api3.geo.admin.ch/rest/services/all/MapServer/identify?
# geometry=2632484.9999999995,1218429.9999999998
# &geometryFormat=geojson
# &geometryType=esriGeometryPoint
# &imageDisplay=1920,1100,96
# &lang=en
# &layers=all:ch.bav.haltestellen-oev,ch.swisstopo.amtliches-strassenverzeichnis
# &limit=10
# &mapExtent=2628437.230258662,1217809.2756955416,2638037.230258662,1223309.2756955416
# &returnGeometry=true
# &sr=2056
# &tolerance=10""".replace('\n','')

# pprint.pprint(requests.get(identifyurl).json())


requesting https://mf-chsdi3.dev.bgdi.ch/ltrea_extend_find/rest/services/all/MapServer/identify?geometry=2577070,1184860&mapExtent=0,0,100,100&imageDisplay=100,100,100&geometryFormat=geojson&geometryType=esriGeometryPoint&lang=de&layers=all:ch.swisstopo.amtliches-strassenverzeichnis&returnGeometry=false&tolerance=500000&sr=2056&where=1%3D1
--> example :1=1: returned error code 400: {"status":"error","code":400,"detail":"The where/layerDefs clause is not valid for ch.swisstopo.amtliches-strassenverzeichnis."}


## where
When using `where` as query param, the filter expression with the attribute(s) contained in the layer specified for identify are directly passed, e.g.
```
&where=plzo+=+'8302 Kloten'
```
**Note: The filter expression must be correctly url-encoded!**

### Examples

In [200]:
for layer, layer_examples in examples.items():
    for example in layer_examples:
        where_params = {
            "branch":BRANCH,
            "paramName":'where',
            "layer":layer,
            "paramValue":encode_param(example)
        }
        get_url(where_params)


requesting https://mf-chsdi3.dev.bgdi.ch/ltboc/rest/services/all/MapServer/identify?geometry=2577070,1184860&mapExtent=0,0,100,100&imageDisplay=100,100,100&geometryFormat=geojson&geometryType=esriGeometryPoint&lang=de&layers=all:ch.bazl.luftfahrthindernis&returnGeometry=false&tolerance=500000&sr=2056&where=bgdi_activesince%20%3C%3D%20%272019-07-30%27
--> example :bgdi_activesince <= '2019-07-30': returned error code 500: <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>500 Internal Server Error</title>
</head><body>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error or
misconfiguration and was unable to complete
your request.</p>
<p>Please contact the server administrator at 
 [no address given] to inform them of the time this error occurred,
 and the actions you performed just before this error.</p>
<p>More information about this error may be available
in the server error log.</p>
</body></html>


requesting https://mf-chsdi3.dev.bgdi.ch/

## layerDefs
When using the query parameter `layerDefs`, the layer on which the filter should be applied is again specified but must be the same as specified in the `layer` parameter for identify, e.g.
```
{"ch.swisstopo.amtliches-strassenverzeichnis": "plzo = '8302 Kloten'"}
```
**Note: also here, the whole expression above must be correctly url encoded!**

### Examples

In [138]:
import json

for layer, layer_examples in examples.items():
#     for example in layer_examples:
#         layerDefsDict = {layer:example}
#         layerDefs_params = {
#             "branch":BRANCH,
#             "layer":layer,
#             "paramName":'layerDefs',
#             "paramValue":encode_param(json.dumps(layerDefsDict))
#         }
#         get_url(layerDefs_params)
    if layer == 'ch.bfe.ladestellen-elektromobilitaet':
        layerDefsDict = {layer:layer_examples[1], layer:layer_examples[2]}
        layerDefs_params = {
            "branch":BRANCH,
            "layer":layer,
            "paramName":'layerDefs',
            "paramValue":encode_param(json.dumps(layerDefsDict))
        }
        get_url(layerDefs_params)


requesting https://mf-chsdi3.dev.bgdi.ch/ltrea_extend_find/rest/services/all/MapServer/identify?geometry=2577070,1184860&mapExtent=0,0,100,100&imageDisplay=100,100,100&geometryFormat=geojson&geometryType=esriGeometryPoint&lang=de&layers=all:ch.bfe.ladestellen-elektromobilitaet&returnGeometry=false&tolerance=500000&sr=2056&layerDefs=%7B%22ch.bfe.ladestellen-elektromobilitaet%22%3A%20%22Longitude%20%3E%207.476%22%7D
--> success! 201 results
{'featureId': 147246,
 'id': 147246,
 'layerBodId': 'ch.bfe.ladestellen-elektromobilitaet',
 'layerName': 'Ladestellen E-Mobilität (im Aufbau)',
 'properties': {'Accessibility': 'Free publicly accessible',
                'AdditionalInfo': '[{"lang": "fr-FR", "value": "A droite '
                                  '\\u00e0 l\'entr\\u00e9e du parking"}]',
                'Address': 'Avenue Max Huber 3 3960 Sierre',
                'AuthenticationModes': ['REMOTE'],
                'Availability': 'Available',
                'ChargingFacilities': ['{"A

### Open questions

- cross-filter functionality in ESRI??lay=100,100,100&geometryFormat=geojson&geometryType=esriGeometryPoint&lang=de&layers=all:ch.swisstopo.amtliches-strassenverzeichnis&returnGeometry=true&tolerance=500000&sr=2056&layerDefs={%22ch.swisstopo.amtliches-strassenverzeichnis%22:%20%22plzo%20=%20%278302%20Kloten%27%20AND%20plzo%20ilike%20%273000%%27%22}

- who is using where other than frontend?
- how to configure in bod to identify layer that is visualized as json but queryable?
- street number? (not included in label, included at all in strassenverz.?)

